## Demo: SingleRun with HasDelayPlugin
The basic steps to set up an OpenCLSim simulation are:
* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

----

This example notebook shows how you can integrate percentual delays in your simulation. We demonstrate how you can use the HasDelayPlugin, and how you can give it a certain delay percentage.

#### 0. Import libraries

In [62]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugins

#### 1. Initialise simpy environment

In [63]:
# setup environment (simulation time needs to match the available weather data)
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes

In [64]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

#### 3. Create objects
##### 3.1. Create site object(s)

In [65]:
# prepare input data for from_site
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)
data_from_site = {"env": my_env,
                  "name": "from_site",
                  "geometry": location_from_site,
                  "capacity": 12,
                  "level": 12
                 }
# instantiate from_site 
from_site = Site(**data_from_site)

# prepare input data for to_site
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
data_to_site = {"env": my_env,
                "name": "to_site",
                "geometry": location_to_site,
                "capacity": 12,
                "level": 0
               }
# instantiate to_site 
to_site = Site(**data_to_site)

##### 3.2. Create vessel object(s)

In [66]:
# prepare input data for vessel_01
data_vessel01 = {"env": my_env,
                 "name": "vessel01",
                 "geometry": location_from_site, 
                 "loading_rate": 1,
                 "unloading_rate": 1,
                 "capacity": 4,
                 "compute_v": lambda x: 10
               }
# instantiate vessel_01 
vessel01 = TransportProcessingResource(**data_vessel01)

##### 3.3 Create activity/activities

In [67]:
# initialise registry
registry = {}
keep_resources = {}

In [68]:
# create delay sequence activities
DelaySequenceActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.SequentialActivity,  # the order is critical!
    ),
    {},
)

DelayWhileActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.WhileActivity,  # the order is critical!
    ),
    {},
)

DelayMoveActivity =  type(
    "TestMoveActivity",
    (
        plugins.HasDelayPlugin,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)

DelayShiftActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.ShiftAmountActivity,  # the order is critical!
    ),
    {},
)

DelayBasicActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.BasicActivity,  # the order is critical!
    ),
    {},
)

In [69]:
# create a list of the sub processes
sub_processes = [
    DelayMoveActivity(
        env=my_env,
        name="sailing empty",
        registry=registry,
        mover=vessel01,
        destination=from_site,
        delay_percentage=10,
    ),
    DelayShiftActivity(
        env=my_env,
        name="load cargo",
        registry=registry,
        processor=vessel01,
        origin=from_site,
        destination=vessel01,
        amount=4,
        duration=10,
        delay_percentage=10,
    ),
    DelayMoveActivity(
        env=my_env,
        name="sailing full",
        registry=registry,
        mover=vessel01,
        destination=to_site,
        delay_percentage=10,
    ),
    DelayShiftActivity(
        env=my_env,
        name="unload cargo",
        registry=registry,
        processor=vessel01,
        origin=vessel01,
        destination=to_site,
        amount=4,
        duration=10,
        delay_percentage=10,
    ),
    DelayBasicActivity(
        env=my_env,
        name="Basic activity",
        registry=registry,
        duration=0,
        additional_logs=[vessel01],
        delay_percentage=10,
    ),
]

# create a 'sequential activity' that is made up of the 'sub_processes'
sequential_activity = DelaySequenceActivity(
    env=my_env,
    name="Single run process",
    registry=registry,
    sub_processes=sub_processes,
    delay_percentage=10,
)

# create a while activity that executes the 'sequential activity' while a stop condition is not met 
while_activity = DelayWhileActivity(
    env=my_env,
    name="While activity",
    registry=registry,
    sub_processes=[sequential_activity],
    condition_event=[{"type": "container", "concept": to_site, "state": "full"}],
    delay_percentage=10,
)

#### 4. Register processes and run simpy

In [70]:
model.register_processes([while_activity])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs

In [71]:
pd.concat([plot.get_log_dataframe(act, [while_activity, sequential_activity, *sub_processes]) for act in sub_processes]).sort_values(by='Timestamp')

,Activity,Timestamp,ActivityState,type,ref
0,sailing empty,1970-01-01 00:00:00.000000,START,NaN,NaN
1,sailing empty,1970-01-01 00:00:00.000000,STOP,NaN,NaN
2,sailing empty,1970-01-01 00:00:00.000000,WAIT_START,plugin,delay
3,sailing empty,1970-01-01 00:00:00.000000,WAIT_STOP,plugin,delay
0,load cargo,1970-01-01 00:00:00.000000,START,NaN,NaN
1,load cargo,1970-01-01 00:00:10.000000,STOP,NaN,NaN
2,load cargo,1970-01-01 00:00:10.000000,WAIT_START,plugin,delay
3,load cargo,1970-01-01 00:00:11.000000,WAIT_STOP,plugin,delay
0,sailing full,1970-01-01 00:00:11.000000,START,NaN,NaN
2,sailing full,1970-01-01 00:15:53.824591,WAIT_START,plugin,delay


##### 5.2 Visualise gantt charts

In [72]:
objects = [while_activity, sequential_activity, vessel01, from_site, to_site]
objects.extend(sub_processes)
plot.get_gantt_chart(objects)